## Testing Windowed GAF Dataset on simple CNN-RNN-DNN model

In [ ]:
# Build the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(168,168,1)),  
    tf.keras.layers.Conv2D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Reshape((-1, 64)), # Reshape the output to a 3D tensor
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Print the model summary 
model.summary()

In [ ]:
# Set the learning rate
learning_rate = 1e-3

# Set the optimizer 
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
# Train the model
history = model.fit(gaf_dataset,epochs=5) # bad idea: it took ETA: 26:03:08 for first epoch

## Test Generator for GAF Windows

In [ ]:
def windowed_GAF_generator(series, window_size, batch_size, shuffle_buffer, stride):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=stride, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    gasf = GramianAngularField(method='summation')
    
    gaf_outputs = []
    for window in dataset:
        window = window.numpy()
        gaf_transformed = gasf.fit_transform(window[:-1].reshape(1, -1))
        gaf_outputs.append((tf.convert_to_tensor(gaf_transformed[0]), tf.convert_to_tensor(window[-1])))

    return gaf_outputs

In [ ]:
gaf_outputs = windowed_GAF_generator(series, window_size, batch_size, shuffle_buffer, stride)

In [ ]:
print(gaf_outputs[0])

## Testing Windowed GAF Function

In [ ]:
def windowed_GAF(series, window_size, batch_size, shuffle_buffer, stride):

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size (note stride argument)
    dataset = dataset.window(window_size + 1, shift=stride, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    
    # Create a GAF transformer
    gasf = GramianAngularField(method='summation')
    
    # Create a function to apply GAF transformation
    def apply_gaf(window):
        return tf.numpy_function(
            lambda win: (gasf.fit_transform(win[:-1].reshape(1, -1)), win[-1]),
            [window],
            (tf.float32, tf.float32),
        )

    # Create tuples with features and labels (Applying GAF transform on each window)
    dataset = dataset.map(apply_gaf)
    
    # Shuffle the windows
    #dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

In [ ]:
dataset_windowed_GAF = windowed_GAF(series_train, window_size, batch_size, shuffle_buffer_size, stride)

## Learning Rate Tuning

In [ ]:
# Get initial weights
init_weights = model.get_weights()

In [ ]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Train the model
history = model.fit(dataset_windowed, epochs=100, callbacks=[lr_schedule])

In [ ]:
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))

# Set the grid
plt.grid(True)

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')

# Set the plot boundaries
plt.axis([1e-8, 1e-3, 0, 100])

In [ ]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

# Reset the weights
model.set_weights(init_weights)

## Testing GAF Window Function

In [ ]:
# Idea, define sampling windows first, then apply GAF transform to each window

In [ ]:
# Apply the GAF transformation to the training and validation sets
gasf_data_train = gasf.fit_transform(series_train.reshape(1, -1))[0]
gasf_data_valid = gasf.fit_transform(series_valid.reshape(1, -1))[0]

In [ ]:
def windowed_dataset_2d(series, window_size, batch_size, shuffle_buffer):
    n_samples = series.shape[1] - window_size
    X = np.empty((n_samples, window_size, series.shape[0]))
    y = np.empty(n_samples)
    
    for i in range(n_samples):
        X[i] = series[:, i:i + window_size].T
        y[i] = series[:, i + window_size]

    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

## h-step forecast

In [ ]:
def h_step_forecast(model, series, window_size, n_forecast):
    y = series
    x_pred = y[-window_size - 1: - 1].reshape(1, window_size, 1)
    y_pred = y[-1].reshape(1, 1, 1)

    y_future = []
    for i in range(n_forecast):
        x_pred = np.append(x_pred[:, 1:, :], y_pred, axis=1)
        y_pred = model.predict(x_pred).reshape(1, 1, 1)
        y_future.append(y_pred.flatten().tolist())

    y_future = np.array(y_future)

    return y_future

h_step = 1000
y_future = h_step_forecast(model, series, window_size, n_forecast = h_step).squeeze()

In [ ]:
# Extend initial forecast with h_step_forecast output
extended = np.concatenate((forecast,y_future))
extended_time = np.concatenate((time_valid,np.arange(time_valid[-1]+1,time_valid[-1]+1+h_step)))
extended_series = np.concatenate((series_valid,np.zeros(h_step)))

plt.figure(figsize=(10, 6))
plt.plot(time_valid, series_valid, label="Actual")
plt.plot(extended_time, extended, label="Predicted")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# ZOOMING INTO LAST 10% OF DATA POINTS
zoom_start_idx = int(0.90 * len(time_valid))

zoom_time_valid = time_valid[zoom_start_idx:]
zoom_series_valid = series_valid[zoom_start_idx:]
zoom_extended_time = extended_time[zoom_start_idx:]
zoom_forecast = extended[zoom_start_idx:]

plt.figure(figsize=(10, 6))
plt.plot(zoom_time_valid, zoom_series_valid, label="Actual")
plt.plot(zoom_extended_time, zoom_forecast, label="Predicted")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.show()

## LSTNet

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
pd.options.mode.chained_assignment = None

from lstnet_tensorflow.layers import SkipGRU
from lstnet_tensorflow.utils import get_training_sequences

class LSTNet():

    def __init__(self,
                 y,
                 forecast_period,
                 lookback_period,
                 filters=100,
                 kernel_size=3,
                 gru_units=100,
                 skip_gru_units=50,
                 skip=1,
                 lags=1,
                 dropout=0,
                 regularizer='L2',
                 regularization_factor=0.01):

        '''
        Implementation of multivariate time series forecasting model introduced in Lai, G., Chang, W. C., Yang, Y.,
        & Liu, H. (2018). Modeling Long- and Short-Term Temporal Patterns with Deep Neural Networks. In "The 41st
        International ACM SIGIR Conference on Research & Development in Information Retrieval" (SIGIR '18).
        Association for Computing Machinery, New York, NY, USA, 95–104. 

        Parameters:
        __________________________________
        y: np.array.
            Time series, array with shape (n_samples, n_targets) where n_samples is the length of the time series
            and n_targets is the number of time series.

        forecast_period: int.
            Number of future time steps to forecast.

        lookback_period: int.
            Number of past time steps to use as input.

        filters: int.
            Number of filters (or channels) of the convolutional layer.

        kernel_size: int.
            Kernel size of the convolutional layer.

        gru_units: list.
            Hidden units of GRU layer.

        skip_gru_units: list.
            Hidden units of Skip GRU layer.

        skip: int.
            Number of skipped hidden cells in the Skip GRU layer.

        lags: int.
            Number of autoregressive lags.

        dropout: float.
            Dropout rate.

        regularizer: str.
            Regularizer, either 'L1', 'L2' or 'L1L2'.

        regularization_factor: float.
            Regularization factor.
        '''

        # Normalize the targets.
        y_min, y_max = np.min(y, axis=0), np.max(y, axis=0)
        y = (y - y_min) / (y_max - y_min)
        self.y_min = y_min
        self.y_max = y_max

        # Extract the input sequences and output values.
        self.X, self.Y = get_training_sequences(y, lookback_period)

        # Save the inputs.
        self.y = y
        self.n_samples = y.shape[0]
        self.n_targets = y.shape[1]
        self.n_lookback = lookback_period
        self.n_forecast = forecast_period

        # Build and save the model.
        self.model = build_fn(
            self.n_targets,
            self.n_lookback,
            filters,
            kernel_size,
            gru_units,
            skip_gru_units,
            skip,
            lags,
            dropout,
            regularizer,
            regularization_factor)

    def fit(self,
            loss='mse',
            learning_rate=0.001,
            batch_size=32,
            epochs=100,
            validation_split=0,
            verbose=1):

        '''
        Train the model.

        Parameters:
        __________________________________
        loss: str, function.
            Loss function, see https://www.tensorflow.org/api_docs/python/tf/keras/losses.

        learning_rate: float.
            Learning rate.

        batch_size: int.
            Batch size.

        epochs: int.
            Number of epochs.

        validation_split: float.
            Fraction of the training data to be used as validation data, must be between 0 and 1.

        verbose: int.
            Verbosity mode: 0 = silent, 1 = progress bar, 2 = one line per epoch.
        '''

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=loss,
        )

        self.model.fit(
            x=self.X,
            y=self.Y,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            verbose=verbose
        )

    def forecast(self, y):
    
        '''
        Generate the forecasts.
        
        Parameters:
        __________________________________
        y: np.array.
            Past values of the time series.

        Returns:
        __________________________________
        df: pd.DataFrame.
            Data frame including the actual values of the time series and the forecasts.
        '''

        # Normalize the targets.
        y = (y - self.y_min) / (self.y_max - self.y_min)
        
        # Generate the multi-step forecasts.
        x_pred = y[- self.n_lookback - 1: - 1, :].reshape(1, self.n_lookback, self.n_targets)   # Last observed input sequence.
        y_pred = y[-1:, :].reshape(1, 1, self.n_targets)                                        # Last observed target value.
        y_future = []                                                                           # Future target values.
        
        for i in range(self.n_forecast):

            # Feed the last forecast back to the model as an input.
            x_pred = np.append(x_pred[:, 1:, :], y_pred, axis=1)

            # Generate the next forecast.
            y_pred = self.model(x_pred).numpy().reshape(1, 1, self.n_targets)

            # Save the forecast.
            y_future.append(y_pred.flatten().tolist())

        y_future = np.array(y_future)

        # Organize the forecasts in a data frame.
        columns = ['time_idx']
        columns.extend(['actual_' + str(i + 1) for i in range(self.n_targets)])
        columns.extend(['predicted_' + str(i + 1) for i in range(self.n_targets)])

        df = pd.DataFrame(columns=columns)
        df['time_idx'] = np.arange(self.n_samples + self.n_forecast)

        for i in range(self.n_targets):

            df['actual_' + str(i + 1)].iloc[: - self.n_forecast] = \
                self.y_min[i] + (self.y_max[i] - self.y_min[i]) * self.y[:, i]

            df['predicted_' + str(i + 1)].iloc[- self.n_forecast:] = \
                self.y_min[i] + (self.y_max[i] - self.y_min[i]) * y_future[:, i]

        # Return the data frame.
        return df.astype(float)


def build_fn(n_targets,
             n_lookback,
             filters,
             kernel_size,
             gru_units,
             skip_gru_units,
             skip,
             lags,
             dropout,
             regularizer,
             regularization_factor):

    '''
    Build the model, see Section 3 in the LSTNet paper.

    Parameters:
    __________________________________
    n_targets: int.
        Number of time series.

    n_lookback: int.
        Number of past time steps to use as input.

    filters: int.
        Number of filters (or channels) of the convolutional layer.

    kernel_size: int.
        Kernel size of the convolutional layer.

    gru_units: list.
        Hidden units of GRU layer.

    skip_gru_units: list.
        Hidden units of Skip GRU layer.

    skip: int.
        Number of skipped hidden cells in the Skip GRU layer.

    lags: int.
        Number of autoregressive lags.

    dropout: float.
        Dropout rate.

    regularizer: str.
        Regularizer, either 'L1', 'L2' or 'L1L2'.

    regularization_factor: float.
        Regularization factor.
    '''

    # Inputs.
    x = tf.keras.layers.Input(shape=(n_lookback, n_targets))

    # Convolutional component, see Section 3.2 in the LSTNet paper.
    c = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(x)
    c = tf.keras.layers.Dropout(rate=dropout)(c)

    # Recurrent component, see Section 3.3 in the LSTNet paper.
    r = tf.keras.layers.GRU(units=gru_units, activation='relu')(c)
    r = tf.keras.layers.Dropout(rate=dropout)(r)

    # Recurrent-skip component, see Section 3.4 in the LSTNet paper.
    s = SkipGRU(units=skip_gru_units, activation='relu', return_sequences=True)(c)
    s = tf.keras.layers.Dropout(rate=dropout)(s)
    s = tf.keras.layers.Lambda(function=lambda x: x[:, - skip:, :])(s)
    s = tf.keras.layers.Reshape(target_shape=(s.shape[1] * s.shape[2],))(s)
    d = tf.keras.layers.Concatenate(axis=1)([r, s])
    d = tf.keras.layers.Dense(units=n_targets, kernel_regularizer=kernel_regularizer(regularizer, regularization_factor))(d)

    # Autoregressive component, see Section 3.6 in the LSTNet paper.
    l = tf.keras.layers.Flatten()(x[:, - lags:, :])
    l = tf.keras.layers.Dense(units=n_targets, kernel_regularizer=kernel_regularizer(regularizer, regularization_factor))(l)

    # Outputs.
    y = tf.keras.layers.Add()([d, l])

    return tf.keras.models.Model(x, y)


def kernel_regularizer(regularizer, regularization_factor):

    '''
    Define the kernel regularizer.

    Parameters:
    __________________________________
    regularizer: str.
        Regularizer, either 'L1', 'L2' or 'L1L2'.

    regularization_factor: float.
        Regularization factor.
    '''

    if regularizer == 'L1':
        return tf.keras.regularizers.L1(l1=regularization_factor)

    elif regularizer == 'L2':
        return tf.keras.regularizers.L2(l2=regularization_factor)

    elif regularizer == 'L1L2':
        return tf.keras.regularizers.L1L2(l1=regularization_factor, l2=regularization_factor)

    else:
        raise ValueError('Undefined regularizer {}.'.format(regularizer))

In [ ]:
import tensorflow as tf

class SkipGRU(tf.keras.layers.Layer):

    def __init__(self,
                 units,
                 p=1,
                 activation='relu',
                 return_sequences=False,
                 return_state=False,
                 **kwargs):

        '''
        Recurrent-skip layer, see Section 3.4 in the LSTNet paper.
        
        Parameters:
        __________________________________
        units: int.
            Number of hidden units of the GRU cell.

        p: int.
            Number of skipped hidden cells.

        activation: str, function.
            Activation function, see https://www.tensorflow.org/api_docs/python/tf/keras/activations.

        return_sequences: bool.
            Whether to return the last output or the full sequence.

        return_state: bool.
            Whether to return the last state in addition to the output.

        **kwargs: See https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRUCell.
        '''

        if p < 1:
            raise ValueError('The number of skipped hidden cells cannot be less than 1.')

        self.units = units
        self.p = p
        self.return_sequences = return_sequences
        self.return_state = return_state
        self.timesteps = None
        self.cell = tf.keras.layers.GRUCell(units=units, activation=activation, **kwargs)

        super(SkipGRU, self).__init__()

    def build(self, input_shape):

        if self.timesteps is None:
            self.timesteps = input_shape[1]

            if self.p > self.timesteps:
                raise ValueError('The number of skipped hidden cells cannot be greater than the number of timesteps.')

    def call(self, inputs):

        '''
        Parameters:
        __________________________________
        inputs: tf.Tensor.
            Layer inputs, 2-dimensional tensor with shape (n_samples, filters) where n_samples is the batch size
            and filters is the number of channels of the convolutional layer.

        Returns:
        __________________________________
        outputs: tf.Tensor.
            Layer outputs, 2-dimensional tensor with shape (n_samples, units) if return_sequences == False,
            3-dimensional tensor with shape (n_samples, n_lookback, units) if return_sequences == True where
            n_samples is the batch size, n_lookback is the number of past time steps used as input and units
            is the number of hidden units of the GRU cell.

        states: tf.Tensor.
            Hidden states, 2-dimensional tensor with shape (n_samples, units) where n_samples is the batch size
            and units is the number of hidden units of the GRU cell.
        '''

        outputs = tf.TensorArray(
            element_shape=(inputs.shape[0], self.units),
            size=self.timesteps,
            dynamic_size=False,
            dtype=tf.float32,
            clear_after_read=False
        )

        states = tf.TensorArray(
            element_shape=(inputs.shape[0], self.units),
            size=self.timesteps,
            dynamic_size=False,
            dtype=tf.float32,
            clear_after_read=False
        )

        initial_states = tf.zeros(
            shape=(tf.shape(inputs)[0], self.units),
            dtype=tf.float32
        )

        for t in tf.range(self.timesteps):

            if t < self.p:
                output, state = self.cell(
                    inputs=inputs[:, t, :],
                    states=initial_states
                )

            else:
                output, state = self.cell(
                    inputs=inputs[:, t, :],
                    states=states.read(t - self.p)
                )

            outputs = outputs.write(index=t, value=output)
            states = states.write(index=t, value=state)

        outputs = tf.transpose(outputs.stack(), [1, 0, 2])
        states = tf.transpose(states.stack(), [1, 0, 2])

        if not self.return_sequences:
            outputs = outputs[:, -1, :]

        if self.return_state:
            states = states[:, -1, :]
            return outputs, states

        else:
            return outputs

In [ ]:
import numpy as np

def get_training_sequences(y, n_lookback):

    '''
    Split the time series into input sequences and output values. These are used for training the model.
    See Sections 3.1 and 3.8 in the LSTNet paper.

    Parameters:
    __________________________________
    y: np.array.
        Time series, array with shape (n_samples, n_targets) where n_samples is the length of the time
        series and n_targets is the number of time series.

    n_lookback: int.
        The number of past time steps used as input.

    Returns:
    __________________________________
    X: np.array.
        Input sequences, array with shape (n_samples - n_lookback, n_lookback, n_targets).

    Y: np.array.
        Output values, array with shape (n_samples - n_lookback, n_targets).
    '''

    X = np.zeros((y.shape[0], n_lookback, y.shape[1]))
    Y = np.zeros((y.shape[0], y.shape[1]))

    for i in range(n_lookback, y.shape[0]):

        X[i, :, :] = y[i - n_lookback: i, :]
        Y[i, :] = y[i, :]

    X = X[n_lookback:, :, :]
    Y = Y[n_lookback:, :]

    return X, Y

In [ ]:
import numpy as np

from lstnet_tensorflow.model import LSTNet
from lstnet_tensorflow.plots import plot

# Generate some time series
N = 500
t = np.linspace(0, 1, N)
e = np.random.multivariate_normal(mean=np.zeros(3), cov=np.eye(3), size=N)
a = 10 + 10 * t + 10 * np.cos(2 * np.pi * (10 * t - 0.5)) + 1 * e[:, 0]
b = 20 + 20 * t + 20 * np.cos(2 * np.pi * (20 * t - 0.5)) + 2 * e[:, 1]
c = 30 + 30 * t + 30 * np.cos(2 * np.pi * (30 * t - 0.5)) + 3 * e[:, 2]
y = np.hstack([a.reshape(-1, 1), b.reshape(-1, 1), c.reshape(-1, 1)])

# Fit the model
model = LSTNet(
    y=y,
    forecast_period=100,
    lookback_period=200,
    kernel_size=3,
    filters=4,
    gru_units=4,
    skip_gru_units=3,
    skip=50,
    lags=100,
)

model.fit(
    loss='mse',
    learning_rate=0.01,
    batch_size=64,
    epochs=100,
    verbose=1
)

# Generate the forecasts
df = model.forecast(y=y)

# Plot the forecasts
fig = plot(df=df)
fig.write_image('results.png', scale=4, height=900, width=700)